# Tutorial 1 - Forecasting with AtmoRep

## Preamble

Training of AtmoRep is a computationally intensive process, typically requiring a couple of weeks on multiple compute nodes of Juwels Booster. 
However, a selection of pretrained AtmoRep models (singleformers and multiformers) has been made available, enabling immediate use in so called zero-shot applications. <br>
Zero-shot applications leverage the intrinsic capabilities of the AtmoRep model and do not require task-specific finetuning or the addition of a specialized network to the core model.
In particular, AtmoRep has demonstrated skill for short-range forecasting/nowcasting, temporal interpolation and data decompression. <br>
In the following, we will explore the forecast application in a zero-shot setting.

#### Objectives
The objectives of this tutorial comprise
- Generation of a six-hour forecast with AtmoRep in inference mode
- Handling and plotting AtmoRep's model output
- Evaluation AtmoRep's forecast capacities

### Prerequisites
To run the following cells of the Jupyter Notebook, please ensure that you fulfill the following requirements:
- Run your JupyterLab on the GPU node of JURECA (use the `dc-gpu`-partition)
- Activate your hackathon kernel (see upper right corner of your Jupyter Notebook)

## Step 1 - Generate a forecast with AtmoRep

### Forecast Inference in AtmoRep

In this section, we will demonstrate how to perform a **global forecast** using AtmoRep. The global data is separated into tiles/neighborhoods using a spatiotemporal grids, the illustration bellow shows how a single tile is processed during a forecast. The data is represented across latitude, longitude, and time dimensions. Forecasting involves predicting future tokens (y) within a subset of this grid based on past observations (x). This procedure allows us to evaluate the model's ability to extrapolate patterns and generate meaningful predictions for unseen temporal data.

![forecast](../img/forecast_1ml.jpg "Data processing during forecast, with x input, y target output")

### To-Do
- [x] get pretrained models 
- [x] ensure that data is linked
- [x] parse user-config to enable user-specific output and model paths

We start by importing some Python packages and classes from the AtmoRep code...

In [ ]:
import os
import time
from pathlib import Path
from atmorep.core.evaluator import Evaluator
from atmorep.config.config import UserConfig

Before continuing, we need to set-up a user-specific configuration. <br>
This procedures provides both, a symbolic link to the directories where pretrained models and the ERA5-data (in zarr stores) are available and the configuration of user-specific output directories for their (custom) models, their inference results (e.g. global forecasts) and their output.

In [ ]:
# !!! ADAPT PATH TO YOUR OWN DIRECTORY!!!
project_dir = Path("/p/project/deepacf/atmo-rep/langguth1/atmo-rep/")                    
user_config = UserConfig.from_path(project_dir)

In [ ]:
print(user_config.__dict__["results"])

In the next step, we are already ready to configure an instance of the `Evaluator`-class. 
This class allows users to perform inference on pretrained (either provided or custom) models. The class loads a pretrained model which is identified via its W&B-ID and then performs the inference step, i.e. only the forward step of the model is executed. The inference output is saved to the `results`-directory under the user-specific project directory. <br> 
Note that the `Evaluator`-class supports various modes for the inference (e.g. global forecasting) as outlined below. 

In [ ]:
model_id='wc5e2i3t' # pre-trained multiforner

mode, options = 'global_forecast', {
                                  'dates' : [[2021, 2, 10, 12], [2021, 2, 11, 0]],    # this corresponds to teh last time step of the sequence 
                                  'token_overlap' : [0, 0],
                                  'forecast_num_tokens' : 2,        # corresponds to a 6h-forecast (=2x3h)
                                  'with_pytest' : False }

For each inference experiment, three key parameters are required:
- The `model_id`, which identifies the pre-trained model used for inference
- The `mode` of inference you wish to perform
- The required attributes, labeled as `options`, necessary to carry out the selected inference mode

In the case of `global_forecast`, Atmorep requires the following options:
- `'dates'`: This is a list of dates. For a single date, it should still be provided as a list containing just that one item. Each date should follow the format `[year, month, day, hour]`, representing the last timestep to be forecasted. Atmorep will load data from 36 hours before this timestamp (inclusive), then start forecasting `forecast_num_tokens` $\times$ 3 hours backwards.
- `'token_overlap'`: Specifies the degree of overlap between tiles (or neighborhoods).
- `'forecast_num_tokens'`: The number of tokens to be forecasted.
- `'with_pytest'`: Enables or disables systematic testing processes. 

After choosing the appropriate mode for our application (global forecasting) and setting the configuration options, we can run the inference by calling the `evaluate`-method of the class object. 
Note that the following process takes a couple of minutes to complete the process.

In [ ]:
os.environ["MASTER_ADDR"] = str(os.environ['SLURM_JOB_NODELIST'])       # AtmoRep needs to know the hostname of the master-node

# run inference
now = time.time()
Evaluator.evaluate( mode, model_id, options, user_config=user_config)
print("time", time.time() - now)

Ensure to track the W&B-ID that is outputted when running the inference. This ID is required to run the subsequent validation (Step 2 of this tutorial).

### To-Dos @Asma:
- [x] Add details about the `global_forecast`-mode, i.e. the required tiling to get a global field with the local neighborhood approach by AtmoRep, the relevance of the `forecast_num_tokens`-parameter etc.
- [x] Provide insight into the different modes of the Evaluator, i.e. what happens with the `forecast`-mode (difference to global_forecast), what is done for temproal interpolation
- [x] Verify what `'num_samples_validate': 128` corresponds to number tiles/neighborhoods
- [x] It would probably also nice to have a visualisation of the masking that is performed in  (global_)forecast-mode (and for others such as temporal interpolation)
- [ ] add visualization for BERT

### Other supported inference modes:
#### BERT Masked Token Mode
Neighborhoods are selected randomly based on dates and geographic locations, in which tokens are randomly masked according to a percentage specified in the configuration file.
```python
mode, options = 'BERT', {'years_val': [2021], 'num_samples_validate': 128, 'with_pytest': True}
```
Options to configure:
- `'years_val'`: Specifies the year from which samples will be drawn.
- `'num_samples_validate'`: Number of validation batches, which is also the number of selected neighborhoods. A neighborhood is a patch in space and time which gets processed together (??? improve the sentence)

#### BERT Forecast Mode
Random neighborhoods are sampled from the global dataset. For each neighborhood, forecasts are generated for a time period set by `forecast_num_tokens` $\times$ 3 hours.
```python
mode, options = 'forecast', {'forecast_num_tokens': 2, 'num_samples_validate': 128, 'with_pytest': True}
```
Previously defined options are used here.

#### Temporal Interpolation Mode
![forecast](../img/temporal_interpol_1ml.jpg "Data processing during forecast, with x input, y target output")

In this mode, AtmoRep fills in temporal gaps by predicting missing data for selected neighborhoods.

```python
mode, options = 'temporal_interpolation', {'idx_time_mask': [5, 6, 7], 'num_samples_validate': 128, 'with_pytest': True}
```
Key option explained:
- `'idx_time_mask'`: Specifies the indices of tokens in a batch of 12 that AtmoRep will attempt to predict.

## Step 2 - Process and validate the AtmoRep output

The AtmoRep output from the inference step is written to zarr-stores. While the zarr Python package provides a straightforward way to read in the data, the multi-dimensional nature of the data is challenging for subsequent processing. For convenience, we make therefore of a small data interface that reads in the data and turns it into xarry DataArray with labelled dimensions. <br>
For more information about xarray, please refer to the [docs](https://docs.xarray.dev/en/latest/getting-started-guide/index.html).

Again, we start by importing an auxiliary class and a method for data retrieval and plotting, respectively.

In [1]:
import numpy as np
import cartopy.crs as ccrs
from jsc_scripts.utils_hackathon.read_atmorep_data import HandleAtmoRepData
from jsc_scripts.utils_hackathon.plotting import plot_global_data 

Next, we initialize the AtmoRep data handler with the W&B-ID of the inference step executed above and with the results directory of AtmoRep.

In [ ]:
input_dir = user_config.__dict__["results"]    
model_id = "ry906z4c"                          # adapt here

ar_data = HandleAtmoRepData(model_id, input_dir)

Then, we can read in the forecasted and the (ground truth) reference data as follows:

In [ ]:
da_fcst = ar_data.read_data("temperature", "pred")
da_ref = ar_data.read_data("temperature", "target")

Let's have a look and check what we obtained:

In [ ]:
da_fcst

In [ ]:
da_ref

Since multi-dimensional arrays are hard to grasp, we will plot the data in the follwoing to get some more insight into AtmoRep's forecasts. 

In [ ]:
%matplotlib inline

# parameters to select the data of interest
tidx = -1
vlevel = 96
offset = -273.15                  # for conveting temperature unit from K to °C

# set some parameters to customize our plots
proj = ccrs.Robinson(central_longitude=0.)      # The projection to display the (global) data
transform = ccrs.PlateCarree()                  # transformation-object to be used when processing the data in the plot-routne (don't change!)
cmap_name = "RdBu_r"                            # colormap 

fcst, ref = da_fcst.isel({"datetime": tidx}).sel({"ml": vlevel}) + offset, \
            da_ref.isel({"datetime": tidx}).sel({"ml": vlevel}) + offset

plot_global_data(fcst, "./test_pred_plot.png",
                 levels=np.arange(-41., 6.), cmap_name=cmap_name, projection=proj, transform=transform)

plot_global_data(ref, "./test_ref_plot.png",
                 levels=np.arange(-41., 6.), cmap_name=cmap_name, projection=proj, transform=transform)

plot_global_data(fcst - ref, "./test_diff_plot.png", 
                 levels=np.arange(-3., 3., 0.1), cmap_name=cmap_name, projection=proj, transform=transform)


### Task 1:
- Plot different variables on different levels
- Describe what you see. Can you explain your findings?

In the following, we will make our analysis more quantitative. For this, we will compute some basic evaluation metrics such as the RMSE and investigate how the results change with lead time. <br>
Again, we don't need to code everything from scratch, but make use of a `Score`-engine that allows computation of several metrics including averaging over user-defined data dimensions. <br>
We start by initialising the `Score`-engine.

In [ ]:
from jsc_scripts.utils_hackathon.metrics import Scores

score_engine = Scores(da_fcst, da_ref, _, ["lat", "lon"])

You may consult the doc-string to get further information on the engine. Alternatively, consult the source code (`/p/project/training2445/shared/atmorep/jsc_scripts/utils_hackathon/metrics.py`)

In [ ]:
import inspect

print(inspect.getdoc(Scores.__init__))

Let's calculate the RMSE and plot it against leadtime:

In [ ]:
rmse = score_engine("rmse")

In [ ]:
print(rmse.sortby(da_fcst["datetime"]))

In [ ]:
from jsc_scripts.utils_hackathon.plotting import plot_metric_line
import xarray as xr

# ML: not yet finished

plot_metric_line(rmse.isel({"ml": 0}), xr.DataArray(0.), xr.DataArray(0.), {"RMSE": "K"}, "./rmse_sample.png", x_coord="datetime", xlabel="leadtime")

### Task 2:
- Plot different variables on different levels
- Go back to the top of the Jupyter Notebook and re-run the forecasting by modifying the maximum leadtime. How does this affect the reults?

## To-Dos @Asma:
- [ ] Revise text and improve
- [ ] Improve instructions in tasks and check what happens when modifying the maximum leadtime via the `forecast_num_tokens`-parameter when evaluating
- [ ] Further/better tasks?
    - [ ] Uncover how data is being rea
- [ ] Create data for more samples -> to be provied in shared-directory
- [ ] Append Jupyter Notebook with evaluation on larger amount of samples (Step 3)

## To-Dos @Michael
- [ ] Revise data handler with separated lead time dimension and init time of forecast
- [ ] Ensure that lead time dimension is consecutive
- [ ] Fix and harmonize doc-strings and used methods